In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# Obs: talvez precise de adaptação em algum lugar dependendo do formato em que
# estiverem as instâncias

class SensorCoverageSolver:
  def __init__(self, cibles: list[tuple[float]], rcapt: float, rcom: float, puits: tuple[float]):
    '''Builds a solver object for the placement of sensors in a grid problem.
    The grid contains several points of interest, namely cibles, and information drainage 
    spots called puits.
    Parameters:
      -cibles: list[tuple[float]]
              2D coordinates for the '''
    self.cibles = cibles
    self.rcapt = rcapt
    self.rcom = rcom
    self.puits = puits
    self.n = len(cibles)
    self.dist_matrix = self.distance_matrix()

  def distance_matrix(self) -> np.ndarray:
    '''Computes the distance between points in the grid, namely cibles.
    Returns:
    - d: np.array, contains 2 by 2 distance between the points.'''
    # we should probably have a more efficient way to compute the distance matrix
    # think meshgrid or pandas' apply function
    d = np.zeros((self.n, self.n))
    for i in range(self.n):
      for j in range(i+1, self.n):
        ci, cj = self.cibles[i], self.cibles[j]
        d[i, j] = np.linalg.norm(ci, cj)
        d[j, i] = d[i, j]
    return d

  def reachable_points(self, i: int):
    '''For a given point of interest (cible) i, computes the set of reachable points,
    within a distance o R_capt.
    Parameters:
    - i: int, index of said point of interest.
    Returns:
    - reachable_set: list of integers, index of reachable points.'''
    reachable_set = np.where(self.dist_matrix[i, :] <= self.rcapt)
    return reachable_set

  def communicable_points(self, i: int):
    '''For a given point of interest (cible) i, computes the set of communicable points,
    within a distance o R_com of i.
    Parameters:
    - i: int, index of said point of interest.
    Returns:
    - com_set: list of integers, index of communicable points.'''
    com_set = np.where(self.dist_matrix[i, :] <= self.rcom)
    return com_set

  def coverage_heuristics(self):
    # Recebe uma instância do problema e retorna os índices onde a gente vai
    # posicionar os sensores

    sensors = set() # Onde os sensores estão de fato
    reachable = set() # Conjunto de pontos com os quais se consegue captar info
    communicable = set()

    # adiciono o ponto mais próximo da puits ao conjunto
    min_dist = np.inf
    start_point = -1
    for i in range(self.n):
      dist = np.linalg.norm(np.array(self.cibles[i]) - np.array(self.puits))
      if dist < min_dist:
        min_dist = dist
        start_point = i

    sensors.add(start_point)
    reachable.update(self.reachable_points(start_point))
    communicable.update(self.communicable_points(start_point))

    # now, we are ready to start the loop
    while len(reachable) < self.n:
      best_point = -1
      best_len_reachable = -1

      for i in communicable - reachable: # pra todo ponto comunicável, mas não reachable
        new_reachable_i = self.reachable_points(i) - reachable # conjunto de novos
        len_new_reachable_i = len(new_reachable_i)

        if len_new_reachable_i > best_len_reachable:
          best_len_reachable = len_new_reachable_i
          best_point = i

      sensors.add(i)
      reachable.update(self.reachable_points(i))
      communicable.update(self.communicable_points(i))

    return sensors # Q: posso ter algum ponto alcançável que não tenha cobrido nenhuma cible?

SyntaxError: incomplete input (ipython-input-30943087.py, line 62)